In [1]:
import os
import time
import numpy as np
from typing import List, Dict, Any, Tuple
from sentence_transformers import SentenceTransformer
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, PodSpec


/home/nacho/Documents/Maestria_IA/PLN2/CEIA-PLN2/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pdfplumber

def extraer_texto_pdf(ruta_pdf):
    texto = ""
    with pdfplumber.open(ruta_pdf) as pdf:
        for pagina in pdf.pages:
            texto += pagina.extract_text() + "\n"
    return texto

# Ejemplo de uso
texto_cv = extraer_texto_pdf("cv_de_pedro_es.pdf")

In [3]:
texto_cv

'IGNACIO TOMÁS DE PEDRO MERMIER\nINFORMACIÓN PERSONAL\nNombre: Ignacio Tomás de Pedro Mermier Nacionalidad: Argentina\nFecha de nacimiento: 14/01/1995 Edad: 29 años\nCorreo electrónico: idepedro@fi.uba.ar\nEXPERIENCIA LABORAL\n2023 - Actualidad — Allegro Microsystems — Analog Design Engineer\nDiseño de circuitos analógicos a nivel de bloque y top para circuitos\nintegrados orientados a sensores magnéticos e inductivos. Verificación por\nmedio de simulaciones por medio de herramientas específicas tales como\nADE, SDE and AMS-DE. Desarrollo de scripts de python para analizar\nresultados y realizar verificaciones a nivel de wafer.\n2021 - 2023 — Allegro Microsystems — Layout Design Engineer\nResponsable del diseño, floorplaning e implementación de layout de circuitos\nde señales mixtas, dentro del sector dedicado a sensores magnéticos\n(MPSBU). Desarrollo de script para mejorar el funcionamiento del PDK.\n2018 - 2023 — Facultad de Ingeniería, Universidad de Buenos Aires — Ayudante de\npri

In [4]:
import re
# Reemplazar los saltos de línea con un espacio
texto_cv_clean = texto_cv.replace('\n', ' ')
texto_cv_clean = texto_cv_clean.replace('\n\n', ' ')
texto_cv_clean = re.sub(r'\s+', ' ', texto_cv_clean).strip()
print(texto_cv_clean[:500])  # Mostrar los primeros 500 caracteres del CV limpio

IGNACIO TOMÁS DE PEDRO MERMIER INFORMACIÓN PERSONAL Nombre: Ignacio Tomás de Pedro Mermier Nacionalidad: Argentina Fecha de nacimiento: 14/01/1995 Edad: 29 años Correo electrónico: idepedro@fi.uba.ar EXPERIENCIA LABORAL 2023 - Actualidad — Allegro Microsystems — Analog Design Engineer Diseño de circuitos analógicos a nivel de bloque y top para circuitos integrados orientados a sensores magnéticos e inductivos. Verificación por medio de simulaciones por medio de herramientas específicas tales com


In [5]:
# Configuración
nombre_indice = "tp2-pln2"
PINECONE_ENVIRONMENT = "us-west1-gcp"
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")


In [9]:

# ================================
# 1. CONFIGURACIÓN INICIAL
# ================================

def configurar_pinecone():
    """
    Configura la conexión con Pinecone usando variables de entorno.
    
    Variables necesarias:
    - PINECONE_API_KEY: Tu clave API de Pinecone
    - PINECONE_ENVIRONMENT: El entorno de Pinecone (ej: 'us-west1-gcp')
    """
    
    # Obtener credenciales desde variables de entorno
    api_key = PINECONE_API_KEY
    environment = PINECONE_ENVIRONMENT
    
    if not api_key:
        raise ValueError("PINECONE_API_KEY no está configurada en las variables de entorno")
    
    # Inicializar Pinecone
    pc = Pinecone(api_key=api_key)
    
    print(f"✅ Pinecone configurado correctamente en {environment}")
    return True

In [10]:
class GeneradorEmbeddings:
    """
    Clase para generar embeddings usando diferentes modelos.
    """
    
    def __init__(self, modelo: str = "sentence-transformers/all-MiniLM-L6-v2"):
        """
        Inicializa el generador de embeddings.
        
        Args:
            modelo (str): Nombre del modelo de Sentence Transformers
        """
        self.modelo_nombre = modelo
        self.modelo = SentenceTransformer(modelo)
        self.dimension = self.modelo.get_sentence_embedding_dimension()
        
        print(f"✅ Modelo '{modelo}' cargado (dimensión: {self.dimension})")
    
    def generar_embedding(self, texto: str) -> List[float]:
        """
        Genera embedding para un texto individual.
        
        Args:
            texto (str): Texto a convertir en embedding
            
        Returns:
            List[float]: Vector de embedding
        """
        embedding = self.modelo.encode(texto)
        return embedding.tolist()
    
    def generar_embeddings_lote(self, textos: List[str]) -> List[List[float]]:
        """
        Genera embeddings para múltiples textos de manera eficiente.
        
        Args:
            textos (List[str]): Lista de textos
            
        Returns:
            List[List[float]]: Lista de vectores de embedding
        """
        embeddings = self.modelo.encode(textos)
        return [emb.tolist() for emb in embeddings]


In [11]:
def crear_indice(nombre_indice: str, dimension: int = 384, metrica: str = "cosine"):
    """
    Crea un nuevo índice en Pinecone.
    
    Args:
        nombre_indice (str): Nombre del índice a crear
        dimension (int): Dimensión de los vectores (depende del modelo de embedding)
        metrica (str): Métrica de similitud ('cosine', 'euclidean', 'dotproduct')
    
    Configuración de infraestructura:
        - Pods: Unidades de cómputo paralelo que procesan las consultas
          • 1 pod = suficiente para desarrollo y proyectos pequeños
          • Más pods = mayor capacidad de consultas simultáneas pero mayor costo
        
        - Réplicas: Copias idénticas del índice distribuidas geográficamente
          • 1 réplica = configuración básica
          • Más réplicas = mayor disponibilidad y tolerancia a fallos
        
        - Tipos de pod disponibles:
          • p1.x1: 1 vCPU, ~5GB RAM (plan gratuito/starter)
          • p1.x2: 2 vCPU, ~10GB RAM
          • p1.x4: 4 vCPU, ~20GB RAM
          • p2.x1: Optimizado para performance
    
    Returns:
        bool: True si se creó exitosamente
    """
    
    # # Verificar si el índice ya existe
    # indices_existentes = pinecone.list_indexes().names()
    
    # if nombre_indice in indices_existentes:
    #     print(f"⚠️  El índice '{nombre_indice}' ya existe")
    #     return True
    

    pc = Pinecone(api_key=PINECONE_API_KEY)
   
        
    # Crear el índice
    if nombre_indice not in pc.list_indexes().names():
        
        pc.create_index(
            name=nombre_indice,
            dimension=dimension,
            metric=metrica,
            spec=ServerlessSpec(
                cloud='aws',
                region="us-east-1"
            )
            # pods=1,  # Pods: Unidades de cómputo que procesan queries. Más pods = mayor throughput pero mayor costo
            # replicas=1,  # Réplicas: Copias del índice para alta disponibilidad. Más réplicas = mayor disponibilidad
            # pod_type="p1.x1"  # Tipo de pod: p1.x1 (gratuito, 1 vCPU), p1.x2 (2 vCPU), p2.x1 (optimizado), etc.
        )
    
        
    else:
        print(f"ℹ️  El índice '{nombre_indice}' ya existe, no se creará uno nuevo")
        return True
    
    # Esperar a que el índice esté listo
    print(f"🔄 Creando índice '{nombre_indice}'...")
    # while nombre_indice not in pinecone.list_indexes():
    #     time.sleep(1)
    
    print(f"✅ Índice '{nombre_indice}' creado exitosamente")
    return True


In [12]:
# ================================
# 3. POBLACIÓN DEL ÍNDICE
# ================================

def poblar_indice_ejemplo(nombre_indice: str, generador: GeneradorEmbeddings):
    """
    Puebla el índice con datos de ejemplo.
    
    Args:
        nombre_indice (str): Nombre del índice de Pinecone
        generador (GeneradorEmbeddings): Instancia del generador de embeddings
    """
    
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Conectar al índice
    indice = pc.Index(nombre_indice)
    
    # Generar los chunks del texto
    print("🔄 Generando chunks del texto...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=360,chunk_overlap=20,
        length_function=len
    )
    chunks = text_splitter.create_documents([texto_cv_clean])
    
    print(f"🔄 Poblando índice con {len(chunks)} chunks...")
    
    # Generar embeddings para todos los textos
    textos = [chunk.page_content for chunk in chunks]
    embeddings = generador.generar_embeddings_lote(textos)
    
    # Preparar datos para inserción en lotes
    vectors_para_insertar = []
    
    for i, doc in enumerate(textos):
        vector_data = {
            "id": str(i),
            "values": embeddings[i],
            "metadata": {
                "texto": str(doc),  # Aquí puedes agregar más metadata si es necesario
                "categoria": "cv",  # Ejemplo de categoría
            }
        }
        vectors_para_insertar.append(vector_data)
    
    # Insertar vectores en el índice
    indice.upsert(vectors=vectors_para_insertar)
    
    # Verificar estadísticas del índice
    estadisticas = indice.describe_index_stats()
    print(f"✅ Índice poblado exitosamente")
    print(f"   📊 Total de vectores: {estadisticas['total_vector_count']}")
    print(f"   📏 Dimensión: {estadisticas['dimension']}")
    
    return True

In [13]:
# ================================
# 4. BÚSQUEDAS EN EL ÍNDICE
# ================================

def buscar_documentos_similares(
    nombre_indice: str, 
    consulta: str, 
    generador: GeneradorEmbeddings,
    top_k: int = 3,
    filtro_metadata: Dict = None
) -> List[Dict[str, Any]]:
    """
    Realiza una búsqueda por similitud en el índice.
    
    Args:
        nombre_indice (str): Nombre del índice de Pinecone
        consulta (str): Texto de consulta para buscar
        generador (GeneradorEmbeddings): Generador de embeddings
        top_k (int): Número de resultados más similares a devolver
        filtro_metadata (Dict): Filtros opcionales por metadata
        
    Returns:
        List[Dict]: Lista de documentos similares con scores
    """
    
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Conectar al índice
    indice = pc.Index(nombre_indice)
    
    # Generar embedding para la consulta
    print(f"🔍 Buscando documentos similares a: '{consulta}'")
    embedding_consulta = generador.generar_embedding(consulta)
    
    # Realizar la búsqueda
    resultados = indice.query(
        vector=embedding_consulta,
        top_k=top_k,
        include_metadata=True,
        filter=filtro_metadata
    )
    
    # Procesar y formatear resultados
    documentos_encontrados = []
    
    print(f"\n📋 Resultados encontrados ({len(resultados['matches'])}):")
    print("=" * 80)
    
    for i, match in enumerate(resultados['matches'], 1):
        documento = {
            "posicion": i,
            "id": match["id"],
            "score": round(match["score"], 4),
            "texto": match["metadata"]["texto"],
        }
        
        documentos_encontrados.append(documento)
        
        # Mostrar resultado formateado
        print(f"{i}. ID: {documento['id']}")
        print(f"   📊 Score: {documento['score']}")
        print(f"   📝 Texto: {documento['texto'][:100]}...")
        print("-" * 80)
    
    return documentos_encontrados



In [14]:
# ================================
# 5. GESTIÓN DEL ÍNDICE
# ================================

def obtener_estadisticas_indice(nombre_indice: str):
    """
    Muestra estadísticas detalladas del índice.
    
    Args:
        nombre_indice (str): Nombre del índice
    """
    
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Conectar al índice
    indice = pc.Index(nombre_indice)
    estadisticas = indice.describe_index_stats()
    
    print(f"\n📊 ESTADÍSTICAS DEL ÍNDICE '{nombre_indice}'")
    print("=" * 50)
    print(f"📦 Total de vectores: {estadisticas.get('total_vector_count', 0)}")
    print(f"📏 Dimensión: {estadisticas.get('dimension', 0)}")
    
    # Mostrar estadísticas por namespace si existen
    if 'namespaces' in estadisticas:
        print(f"🏷️  Namespaces:")
        for namespace, stats in estadisticas['namespaces'].items():
            print(f"   - {namespace}: {stats.get('vector_count', 0)} vectores")

In [15]:
def buscar_con_filtros_ejemplo(nombre_indice: str, generador: GeneradorEmbeddings):
    """
    Demuestra búsquedas con filtros de metadata.
    
    Args:
        nombre_indice (str): Nombre del índice
        generador (GeneradorEmbeddings): Generador de embeddings
    """
    
    print("\n🔍 EJEMPLO DE BÚSQUEDAS CON FILTROS")
    print("=" * 50)
    
    # Búsqueda 1: Sin filtros
    print("\n1️⃣:")
    buscar_documentos_similares(
        nombre_indice, 
        "machine learning", 
        generador,
        top_k=3
    )
    
    # Búsqueda 2: Con filtro por categoría
    print("\n2️⃣:")
    buscar_documentos_similares(
        nombre_indice,
        "processamiento de datos",
        generador,
        top_k=2,
    )
    
    # Búsqueda 3: Con filtro por fecha
    print("\n3️⃣:")
    buscar_documentos_similares(
        nombre_indice,
        "redes neuronales",
        generador,
        top_k=5,
    )


In [16]:
def limpiar_indice_completo(nombre_indice: str):
    """
    Elimina todos los vectores del índice.
    
    Args:
        nombre_indice (str): Nombre del índice a limpiar
    """
    
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Conectar al índice
    indice = pc.Index(nombre_indice)
    
    print(f"🧹 Limpiando índice '{nombre_indice}' completamente...")
    indice.delete(delete_all=True)
    
    print("✅ Índice limpiado exitosamente")


In [17]:
limpiar_indice_completo(nombre_indice)

🧹 Limpiando índice 'tp2-pln2' completamente...
✅ Índice limpiado exitosamente


In [19]:
"""
Ejecuta un ejemplo completo de uso de Pinecone:
1. Configuración
2. Creación del índice
3. Población con datos
4. Búsquedas de ejemplo
5. Limpieza (opcional)
"""

try:
    print("🚀 INICIANDO EJEMPLO DE PINECONE")
    print("=" * 50)
    
    # 1. Configurar conexión
    configurar_pinecone()
    
    # 2. Inicializar generador de embeddings
    generador = GeneradorEmbeddings()
    
    # 3. Crear índice
    crear_indice(nombre_indice, dimension=generador.dimension)
    
    # 4. Poblar índice con datos de ejemplo
    poblar_indice_ejemplo(nombre_indice, generador)
    
    # 5. Mostrar estadísticas
    obtener_estadisticas_indice(nombre_indice)
    
    # 6. Realizar búsquedas de ejemplo
    buscar_con_filtros_ejemplo(nombre_indice, generador)
    
    # 7. Búsqueda personalizada
    print("\n🎯 BÚSQUEDA PERSONALIZADA")
    print("=" * 30)
    consulta_personalizada = "Nacionalidad"
    resultados = buscar_documentos_similares(
        nombre_indice, 
        consulta_personalizada, 
        generador,
        top_k=2
    )
    
    print(f"\n✅ EJEMPLO COMPLETADO EXITOSAMENTE")
    print(f"📁 Índice '{nombre_indice}' está listo para usar")
    
    # Opcional: Comentar la siguiente línea si quieres mantener el índice
    # eliminar_indice(nombre_indice)
    
except Exception as e:
    print(f"❌ Error durante la ejecución: {str(e)}")
    raise


🚀 INICIANDO EJEMPLO DE PINECONE
✅ Pinecone configurado correctamente en us-west1-gcp
✅ Modelo 'sentence-transformers/all-MiniLM-L6-v2' cargado (dimensión: 384)
ℹ️  El índice 'tp2-pln2' ya existe, no se creará uno nuevo
🔄 Generando chunks del texto...
🔄 Poblando índice con 22 chunks...
✅ Índice poblado exitosamente
   📊 Total de vectores: 22
   📏 Dimensión: 384

📊 ESTADÍSTICAS DEL ÍNDICE 'tp2-pln2'
📦 Total de vectores: 22
📏 Dimensión: 384
🏷️  Namespaces:
   - : 22 vectores

🔍 EJEMPLO DE BÚSQUEDAS CON FILTROS

1️⃣:
🔍 Buscando documentos similares a: 'machine learning'

📋 Resultados encontrados (3):
1. ID: 11
   📊 Score: 0.3452
   📝 Texto: (EAMTA) - Machine Learning en VLSI - Presentador: Dr. Ing. Pedro Julian - Track básico VLSI - 2017 -...
--------------------------------------------------------------------------------
2. ID: 5
   📊 Score: 0.2205
   📝 Texto: INASE, incluyendo manejo de FrontEnd, BackEnd y administración de la Base de Datos, utilizando Djang...
--------------------------

In [16]:
from langchain_pinecone import PineconeVectorStore  
from langchain_community.embeddings import HuggingFaceEmbeddings
pc = Pinecone(api_key=PINECONE_API_KEY)
    
# Conectar al índice
indice = pc.Index(nombre_indice)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
text_field = "texto"  
vectorstore = PineconeVectorStore(  
    indice, embeddings, text_field  
) 

/tmp/ipykernel_113854/50843343.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
query = "Machine Learning y Deep Learning"  
vectorstore.similarity_search(  
    query,  # our search query  
    k=3  # return 3 most relevant docs  
)  

[Document(id='25', metadata={'categoria': 'cv'}, page_content='(EAMTA) - Machine Learning en VLSI - Presentador: Dr. Ing. Pedro Julian - Track básico VLSI - 2017 - Presentador: Dr. Ing. Pedro Julián - Diseño de circuitos integrados para aplicaciones implacables'),
 Document(id='24', metadata={'categoria': 'cv'}, page_content='Laboratorio de Control de Accionamientos, Tracción y Potencia FORMACIÓN COMPLEMENTARIA Escuela Argentina de Micro y Nano Tecnología y Aplicaciones (EAMTA) - Machine Learning en VLSI - Presentador:'),
 Document(id='16', metadata={'categoria': 'cv'}, page_content='Ingeniería Universidad de Buenos Aires — 2024 - Actualidad Título: Master en Inteligencia Artificial Embebida. Facultad de Ingeniería Universidad de Buenos Aires — 2014 - 2019 Título: Ingeniero')]

In [18]:
from groq import Groq           # Cliente oficial de Groq para acceso a LLMs
from langchain_groq import ChatGroq              # Integración LangChain-Groq

model = "llama3-8b-8192"  # Modelo Groq seleccionado por el usuario
groq_chat = ChatGroq(
    groq_api_key=GROQ_API_KEY,     # Clave API para autenticación
    model_name=model,              # Modelo seleccionado por el usuario
    temperature=0.7,               # Creatividad de las respuestas (0=determinista, 1=creativo)
    max_tokens=1000,               # Máximo número de tokens en la respuesta
)

In [21]:
from langchain.chains import RetrievalQA  

query = "Todas las nacionalidades de Ignacio"


qa = RetrievalQA.from_chain_type(  
    llm=groq_chat,  
    chain_type="stuff",  
    retriever=vectorstore.as_retriever()  
)  

# qa_cv = RetrievalQA.from_chain_type(  
#     llm=groq_chat,  
#     chain_type="stuff",  
#     retriever=vectorstore_cv.as_retriever()  
# ) 
result = qa.invoke(query)

print(result['result'])

Según la información proporcionada, Ignacio Tomás de Pedro Mermier tiene la nacionalidad Argentina.
